## Jpype

Dùng cách này gọi `dbapi2` của `jpype` sẽ có phương thức `cursor()` -> pandas có thể tạo df từ đây

In [ ]:
import jpype
import jpype.imports
import jpype.dbapi2
classpaths = ["/opt/phoenix/phoenix-client-embedded-hbase-2.5.jar", "/opt/phoenix/lib/*"]
for classpath in classpaths:
  jpype.addClassPath(classpath)
jpype.startJVM()

In [2]:
conn = jpype.dbapi2.connect(dsn="jdbc:phoenix:localhost", driver="org.apache.phoenix.jdbc.PhoenixDriver")

In [ ]:
import pandas as pd

# Corrected SQL query
query = """
SELECT chi_tiet.*
FROM chi_tiet, chiso_version
WHERE chi_tiet.version = (
	SELECT MAX(version) FROM chiso_version
)
"""

# Execute the query and create a DataFrame
df = pd.read_sql(query, conn)

In [ ]:
import streamlit

streamlit.write(df)

## Py4j
- set up right Java systemwide version
- It worth to notice that py4j connection after calling `DriverManager.getConnection` does not have `cursor()` method which a DB api normally has (which result to `pandas` can not create df from)

In [ ]:
from py4j.java_gateway import JavaGateway

# Open JVM interface with the JDBC Jar
jdbc_jar_path = '/opt/phoenix/*'
log4j_jar_path = '/opt/phoenix/lib/*'
gateway = JavaGateway.launch_gateway(classpath=f"{jdbc_jar_path}:{log4j_jar_path}")

# Load the JDBC Jar
jdbc_class = "org.apache.phoenix.jdbc.PhoenixDriver"
gateway.jvm.Class.forName(jdbc_class)

# Initiate connection
jdbc_uri = "jdbc:phoenix:localhost"
con =  gateway.jvm.java.sql.DriverManager.getConnection(jdbc_uri)

# Execute query
query_chiso_version = 'select max(version) from chiso_version'
stmt = con.createStatement()
rs = stmt.executeQuery(query_chiso_version)

while rs.next():
    print(rs.getString(1))
    # print(rs.getString(2))
    # print(rs.getString(3))
    # print(rs.getString(4)
  